#### Import Libraries

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

####Launch Spark Context

In [2]:
import findspark
findspark.init()
print findspark.find()
import pyspark
conf = (pyspark.SparkConf()
    .setMaster('local')
    .setAppName('pyspark')
    .set("spark.executor.memory", "2g"))
sc = pyspark.SparkContext(conf=conf)

/usr/local/opt/apache-spark/libexec


####Check child execution processes

In [3]:
import sys
rdd = sc.parallelize(xrange(10),10)
rdd.map(lambda x: sys.version).collect()

['2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x8

In [4]:
from pyspark.sql import SQLContext
sqlsc=SQLContext(sc)

In [38]:
pandas_df = pd.read_csv('../dataset/tweets.csv')
#pandas_df.head(5)
alltweetsdf = sqlsc.createDataFrame(pandas_df)
alltweetsdf.show(5)


+----------+--------------------+-----------+-------------+--------------------+--------------------+-----------------+-------------+
|Unnamed: 0|        bounding_box|       city|      country|                text|                 tid|        timestamp|          uid|
+----------+--------------------+-----------+-------------+--------------------+--------------------+-----------------+-------------+
|         0|{u'type': u'Polyg...|  BrasÃ­lia|       Brasil|    @1menandro da mt|6.700545073599447E17|1.448588418222E12|  7.0561688E7|
|         1|{u'type': u'Polyg...|  Las Vegas|United States|snapchat saw it f...|6.700545067264286...|1.448588418071E12|3.237387799E9|
|         2|{u'type': u'Polyg...| Montevideo|      Uruguay|@JessGlynne i lov...|6.700545075612508...| 1.44858841827E12| 5.08050263E8|
|         3|{u'type': u'Polyg...|Miami Beach|United States|@VelozaJefferson ...|6.700545075612713E17| 1.44858841827E12| 4.04523779E8|
|         4|{u'type': u'Polyg...|  Asheville|United States|I l

In [36]:
alltweetsdf.printSchema()

root
 |-- Unnamed: 0: string (nullable = true)
 |-- bounding_box: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- text: string (nullable = true)
 |-- tid: double (nullable = true)
 |-- timestamp: double (nullable = true)
 |-- uid: double (nullable = true)



####Select US tweets

In [56]:
#alltweetsdf.select("country").show()
alltweetsdf.groupBy("country").count().show()

+--------------------+-----+
|             country|count|
+--------------------+-----+
|                 NaN|    5|
|            Hrvatska|    1|
|           Guatemala|   46|
|            CuraÃ§ao|    1|
|            Barbados|    8|
|            Portugal|   80|
|        Saint-Martin|    1|
|         Deutschland|   25|
|              Polska|    4|
|       United States|16862|
|British Virgin Is...|    5|
|            Colombia| 1068|
|              Canada|  644|
|            RomÃ¢nia|    3|
|             EspaÃ±a|  264|
|             Bahamas|   11|
|           Nicaragua|   39|
|             Bolivia|    9|
|          1658744474|    1|
|               Chile|  627|
+--------------------+-----+
only showing top 20 rows



In [85]:
ustweetsdf = alltweetsdf.filter(alltweetsdf['country'] == "United States" )
#ustweetsdf.cache()
print "Count of All tweets =", alltweetsdf.count()
print "Count of US tweets:", ustweetsdf.count()
ustweetsdf.show(5)


Count of All tweets = 45165
Count of US tweets: 16862
+----------+--------------------+------------+-------------+--------------------+--------------------+-----------------+-------------+
|Unnamed: 0|        bounding_box|        city|      country|                text|                 tid|        timestamp|          uid|
+----------+--------------------+------------+-------------+--------------------+--------------------+-----------------+-------------+
|         1|{u'type': u'Polyg...|   Las Vegas|United States|snapchat saw it f...|6.700545067264286...|1.448588418071E12|3.237387799E9|
|         3|{u'type': u'Polyg...| Miami Beach|United States|@VelozaJefferson ...|6.700545075612713E17| 1.44858841827E12| 4.04523779E8|
|         4|{u'type': u'Polyg...|   Asheville|United States|I love u https://...|6.700545060471234...|1.448588417909E12|2.588127421E9|
|         8|{u'type': u'Polyg...|       Boise|United States|I have $1 left #B...|6.700545079512309...|1.448588418363E12|2.980365728E9|
|

In [112]:
#  http://s3-eu-west-1.amazonaws.com/files.figshare.com/360592/Data_Set_S1.txt
happiness_dict = {}
def read_sentiment_wordList(wordlist_file):
    inputfile = open(wordlist_file, 'r')
    
    filetext = inputfile.readlines() 
    
    for line in filetext[4:]:
        rowfields = line[:-1].split('\t')
        happiness_dict[rowfields[0]] = float(rowfields[1])


    return happiness_dict

read_sentiment_wordList('./Data_Set_S1.txt')

print happiness_dict['celebrate']

27.0


In [87]:
#https://github.com/OhLookCake/EmoticonSentiment/blob/master/scripts/scoreEmoticons.R
emoticonlist =["\\:\\-\\)","\\:\\)","\\=\\)", 
               "\\:\\-D","\\:D","8\\-D","8D","x\\-D","xD","X\\-D","XD", 
               "\\:\\-\\(","\\:\\(", "\\:\\-\\|","\\:\\|", "\\:\\'\\-\\(","\\:\\'\\(\\)",
               "\\:\\'\\-\\)","\\:\\'\\)", 
               "\\:\\-o","\\:\\-O","\\:o","\\:O", 
               "o_O","o\\.O","O_o","O\\.o", "\\:\\*","\\;\\-\\)","\\;\\)", "\\%\\-\\)",
               "\\<3","\\<\\/3" ]
print emoticonlist[0]

\:\-\)


In [90]:
#https://github.com/bear/python-twitter/blob/master/twitter/parse_tweet.py
emoticons_pos = ["*O","*-*","*O*","*o*","* *",
                ":P",":D",":d",":p",
                ";P",";D",";d",";p",
                ":-)",";-)",":=)",";=)",
                ":<)",":>)",";>)",";=)",
                "=}",":)","(:;)",
                "(;",":}","{:",";}",
                "{;:]",
                "[;",":')",";')",":-3",
                "{;",":]",
                ";-3",":-x",";-x",":-X",
                ";-X",":-}",";-=}",":-]",
                ";-]",":-.)",
                "^_^","^-^"]

emoticons_neg = [":(",";(",":'(",
                "=(","={","):",");",
                ")':",")';",")=","}=",
                ";-{{",";-{",":-{{",":-{",
                ":-(",";-(",
                ":,)",":'{",
                "[:",";]"
                ]
print emoticons_pos[1]   

*-*


#### Perform Natural Language Processing

In [ ]:
#tokenize
from pattern.en import parse
from pattern.en import pprint
from pattern.vector import stem, PORTER, LEMMA
punctuation = list('.,;:!?()[]{}`''\"@#$^&*+-|=~_')


In [ ]:
from sklearn.feature_extraction import text 
stopwords=text.ENGLISH_STOP_WORDS

In [ ]:
#create column